In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_martin  as vmm 
from vessel_manoeuvring_models.models.vmm import ModelSimulator
from example_ship2 import ship_parameters, df_parameters


if os.name == 'nt':
    plt.style.use('../docs/book/book.mplstyle')  # Windows
    
from vessel_manoeuvring_models.visualization.plot import track_plot, plot

In [ ]:
from vessel_manoeuvring_models.extended_kalman_vmm import ExtendedKalman
from vessel_manoeuvring_models.models.regression import MotionRegression

## Load test

In [ ]:
id=22773
df, units, meta_data = mdl.load(dir_path = '../data/raw', id=id)
df.index = df.index.total_seconds()
df.index-=df.index[0]
df['x0']-=df.iloc[0]['x0']
df['y0']-=df.iloc[0]['y0']
#df['psi']-=df.iloc[0]['psi']


In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

## Filtering

### Lowpass filter

In [ ]:
from numpy import cos as cos
from numpy import sin as sin
from vessel_manoeuvring_models.data.lowpass_filter import lowpass_filter

df_lowpass = df.copy()
t = df_lowpass.index
ts = np.mean(np.diff(t))
fs = 1/ts

position_keys = ['x0','y0','psi']
for key in position_keys:
    df_lowpass[key] = lowpass_filter(data=df_lowpass[key], fs=fs, cutoff=1, order=1)

df_lowpass['x01d_gradient'] = x1d_ = np.gradient(df_lowpass['x0'], t)
df_lowpass['y01d_gradient'] = y1d_ = np.gradient(df_lowpass['y0'], t)
df_lowpass['r'] = r_ = np.gradient(df_lowpass['psi'], t)

psi_ = df_lowpass['psi']

df_lowpass['u'] = x1d_*cos(psi_) + y1d_*sin(psi_)
df_lowpass['v'] = -x1d_*sin(psi_) + y1d_*cos(psi_)

velocity_keys = ['u','v','r']
for key in velocity_keys:
    df_lowpass[key] = lowpass_filter(data=df_lowpass[key], fs=fs, cutoff=1, order=1)

In [ ]:
data = df.copy()
data['thrust'] = data['Prop/PS/Thrust'] + data['Prop/SB/Thrust']

data['u'] = df_lowpass['u']
data['v'] = df_lowpass['v']
data['r'] = df_lowpass['r']
data=data.iloc[200:-100]
data.index-=data.index[0]

### Extended Kalman filter

In [ ]:
parameters = df_parameters["prime"].copy()
ek = ExtendedKalman(vmm=vmm, 
                    parameters=parameters, 
                    ship_parameters=ship_parameters)


#### Simulate with system model

In [ ]:
E = np.array([
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1],
    ],
)

t = np.linspace(0,10,100)
data_ = pd.DataFrame(index=t)
data_['delta'] = 0.0
data_['thrust'] = 20
data_['x0'] = 0
data_['y0'] = 0
data_['psi'] = 0
data_['u'] = 2
data_['v'] = 0
data_['r'] = 0

ek.simulate(data=data, input_columns=['delta','thrust'], 
            E=E).tail()

In [ ]:
data_frames = {'data':data, 'sim':ek.df_simulation}

fig,ax=plt.subplots()
styles = {
    'Mesurement' : {
        'linestyle' : '',
        'marker' : '.',
        'ms' : 1,
        'zorder':-10,
    },
    
    'Kalman filter' : {
        'lw' : 2,
    },
    
    
}

for label,df_ in data_frames.items():
    track_plot(
            df=df_,
            lpp=ship_parameters["L"],
            beam=ship_parameters["B"],
            ax=ax,
            label=label,
            plot_boats=True,
            **styles.get(label,{})
        );
ax.legend()

plot(data_frames, keys=ek.df_simulation.columns);

### Extended Kalman filter and RTS smoother

In [ ]:
Cd = np.array([
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
])

E = np.array([
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1],
    ],
)

P_prd = np.diag([0.1, 0.1, np.deg2rad(0.01), 0.01, 0.01, np.deg2rad(0.01)])
Qd = np.diag([0.01, 0.01, np.deg2rad(0.01)])  #process variances: u,v,r

error_max_pos = 0.1
sigma_pos = error_max_pos/3
variance_pos = sigma_pos**2

error_max_psi = np.deg2rad(0.1)
sigma_psi = error_max_psi/3
variance_psi = sigma_psi**2

Rd = np.diag([variance_pos, variance_pos, variance_psi])

ek.filter(
        data=data, P_prd=P_prd, Qd=Qd, Rd=Rd, E=E, Cd=Cd, 
        input_columns=['delta','thrust'],
    )


ek.smoother();

In [ ]:
dataframes = {
    'Mesurement' : data,
    #'Kalman filter' : ek.df_kalman,
    'RTS': ek.df_smooth,
}

styles = {
    'Mesurement' : {
        'linestyle' : '',
        'marker' : '.',
        'ms' : 1,
        'zorder':-10,
    },
    
    'Kalman filter' : {
        'lw' : 2,
    },
    
    
}

plot(dataframes = dataframes, 
     fig_size=(10,10), 
     styles = ['r-','g-','b-'],
     keys=['x0','y0','psi','u','v','r','u1d','v1d','r1d']);

## Inverse dynamics PIT

In [ ]:
added_masses = df_parameters.groupby(by='state').get_group('dot')['prime'].dropna().to_dict()
added_masses

In [ ]:
regression = MotionRegression(vmm=vmm, data=ek.df_smooth, added_masses=added_masses, ship_parameters=ship_parameters, 
                             base_features=[u,v,r,delta,thrust], exclude_parameters={'Xthrust':1})

#regression = MotionRegression(vmm=vmm, data=ek.df_smooth, added_masses=added_masses, ship_parameters=ship_parameters, 
#                              base_features=[u,v,r,delta,thrust])

In [ ]:
ship_parameters_prime = regression.ps.prime(regression.ship_parameters)

In [ ]:
regression.added_masses

In [ ]:
ship_parameters_prime['m'] - regression.added_masses['Xudot']

In [ ]:
regression.show()

In [ ]:
model = regression.create_model(control_keys=['delta','thrust'])

In [ ]:
result_regression = model.simulate(df_=ek.df_smooth)

In [ ]:
result_regression.track_plot(compare=True);

In [ ]:
result_regression.plot(compare=True);

In [ ]:
X = regression.X_X

#X = sm.add_constant(X)

model_N = sm.OLS(regression.y_X, X, hasconst=True)
result = model_N.fit()
result.summary()

In [ ]:
coefficients = regression.diff_eq_X.get_coefficients()
coefficients

In [ ]:
coefficients0 = list(set(coefficients) - set([p.Xthrust]))
subs = [(coeff,0) for coeff in coefficients0]

In [ ]:
eq_exclude = regression.diff_eq_X.acceleration_equation.subs(subs)
eq_exclude

In [ ]:
subs = {value: key for key, value in p.items()}
lambda_exclude = lambdify(eq_exclude.rhs.subs(subs))
lambda_exclude

In [ ]:
X